In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ingest import ingest_gpx_from_path_to_db, Tracks, Session

In [ ]:
session = Session()

In [ ]:
session.commit()

In [ ]:
query = session.query(Tracks).order_by(Tracks.name)

In [ ]:
for track in query:
    print(track.id, track.name, track.time, track.geometry)

In [ ]:
ingest_gpx_from_path_to_db("./local_data/")